In [1]:
cd ..

/home/dmitrii.shubin/Desktop/fraud_detection-main/modeling


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import PolynomialFeatures
import os

In [8]:
def feature_engineering(df,features_interaction):
    
    df['diff'] = df['Time'].diff()
    
    df_sub = df[features_interaction]
    df = df.drop(features_interaction,axis=1)
    
    
    #get feature interactions
    poly = PolynomialFeatures(2, interaction_only=True, include_bias=False)
    poly.fit(df_sub)
    columns = poly.get_feature_names()
    df_sub = poly.transform(df_sub)
    df_sub = pd.DataFrame(df_sub,columns=columns)
    
    #drop constant columns
    df_sub.loc[:, (df_sub != df_sub.iloc[0]).any()] 
    
    df = pd.concat([df_sub,df],axis=1)

    
    return df

# Field for global variables

In [4]:
DATA_PATH = './data/raw/'
PROCESSED_DATA_PATH = './data/processed/'

N_SPLITS = 5
N_TRAIN = 47000
N_VAL = 20000
N_TEST = 29807

DROPLIST = ['Time','Class']
FEATURES_FOR_INTERACTION = ['V1','V2','V3','V4','V5','V7','V9','V10','V11','V12','V13','V16','V17','V18']
files = os.listdir(DATA_PATH)

# Load the data

In [5]:
for index,file in enumerate(files):
    
    if index == 0:
        df = pd.read_csv(DATA_PATH+file)
    else:
        temp = pd.read_csv(DATA_PATH+file)
        df = df.append(temp,axis=0)

df = df.sort_values(by='Time')
df.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
original_features = df.drop(['Time','Class'],axis=1).columns.to_list()
original_features = pd.DataFrame(original_features,columns=['0'])
original_features.to_csv('./experiments/original_features/features.csv',index=False)

man_features = pd.DataFrame(FEATURES_FOR_INTERACTION,columns=['0'])
man_features.to_csv('./experiments/man_features/features.csv',index=False)

In [9]:
df = feature_engineering(df=df,features_interaction=FEATURES_FOR_INTERACTION)
df.head(5)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,diff
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.239599,0.363787,0.090794,-0.551600,-0.617801,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,NaN
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.078803,-0.255425,-0.166974,1.612727,1.065235,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,0.0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,0.791461,-1.514654,0.207643,0.624501,0.066084,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,1.0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,0.237609,-1.387024,-0.054952,-0.226487,0.178228,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.592941,0.817739,0.753074,-0.822843,0.538196,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,1.0


In [10]:
corr = df.corr()
corr = corr.abs() #abs to estimate the absolute importance towards the target
corr.style.background_gradient(cmap='coolwarm')

In [11]:
orig_and_gen_fetaures = df.drop(['Time','Class'],axis=1).columns.to_list()
orig_and_gen_fetaures = pd.DataFrame(orig_and_gen_fetaures,columns=['0'])
orig_and_gen_fetaures.to_csv('./experiments/orig_and_gen_features/features.csv',index=False)

# Create split tables

In [12]:
# Test subset
df_test = df.iloc[-N_TEST:,:]
df_test.to_csv(PROCESSED_DATA_PATH+'test.csv',index=False)

df = df.iloc[:-N_TEST,:]

In [13]:
for split in range(N_SPLITS):
    
    df_train = df.iloc[N_TRAIN*split:N_TRAIN*(split+1),:]
    df_val = df.iloc[N_TRAIN*(split+1):N_TRAIN*(split+1)+N_VAL,:]
    
    df_train.to_csv(PROCESSED_DATA_PATH+f'{split}_train.csv',index=False)
    df_val.to_csv(PROCESSED_DATA_PATH+f'{split}_val.csv',index=False)